# Import and init

In [73]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [74]:
filepath = 'wiki_giga_2024_50_MFT20_vectors_seed_123_alpha_0.75_eta_0.075_combined.txt'


# Read file

## Practice checks

In [75]:
# First, read the first few lines to check what the input looks like
with open(filepath, 'r') as f:
    first_txt = f.read(1500) + '...'

print(first_txt)

the -0.383396 -0.480674 -0.27414099999999997 0.132697 0.06488499999999997 -0.092489 -0.0029909999999999937 0.168918 0.42769 0.032062000000000035 0.600652 0.0045550000000000035 0.221568 -0.106817 0.339955 -0.24816500000000002 -0.06553399999999998 0.16503099999999998 -0.098915 -0.26706 -0.351612 -0.595836 -0.748328 -0.262191 -0.400183 0.306477 0.13247 0.149532 -0.462982 0.436406 -0.328406 -0.11229700000000001 -0.318158 -0.586177 -0.24174600000000002 -0.38285899999999995 -0.433786 5.955204999999999 0.137239 -0.064516 0.456522 -0.019239000000000006 0.260624 -0.293577 0.19897499999999999 0.277653 0.164552 0.40649 -0.227907 -0.400316
, 0.151841 0.07100100000000001 -0.306629 -0.24564200000000003 0.163437 0.343526 -0.635576 -0.08061000000000007 -0.281144 0.01024799999999998 -0.100773 0.041249999999999995 -0.284503 0.45056999999999997 -0.44594599999999995 -0.058714999999999996 -0.06081500000000001 0.006295999999999996 -0.35699400000000003 -0.10028200000000001 0.270828 0.40327899999999994 0.3603

In [76]:
# Check a sample of 10 sentences
glove_vectors, glove_tokens = [], []

with open(filepath, 'r') as f:
    for i in range(10):
        line = f.readline()
        if i == 0:
            dim_v_full = len(line.split(' '))
            dim_v = dim_v_full - 1
            print(dim_v_full)
        full_line = line.strip().split(' ')
        glove_tokens.append(full_line[0])
        glove_vectors.append(full_line[1:])

print(len(glove_tokens), glove_tokens)
print(len(glove_vectors), glove_vectors)
dim_v = len(glove_vectors[0])
print([len(i) for i in glove_vectors])

51
10 ['the', ',', '.', 'of', 'to', 'and', 'in', 'a', '"', '-']
10 [['-0.383396', '-0.480674', '-0.27414099999999997', '0.132697', '0.06488499999999997', '-0.092489', '-0.0029909999999999937', '0.168918', '0.42769', '0.032062000000000035', '0.600652', '0.0045550000000000035', '0.221568', '-0.106817', '0.339955', '-0.24816500000000002', '-0.06553399999999998', '0.16503099999999998', '-0.098915', '-0.26706', '-0.351612', '-0.595836', '-0.748328', '-0.262191', '-0.400183', '0.306477', '0.13247', '0.149532', '-0.462982', '0.436406', '-0.328406', '-0.11229700000000001', '-0.318158', '-0.586177', '-0.24174600000000002', '-0.38285899999999995', '-0.433786', '5.955204999999999', '0.137239', '-0.064516', '0.456522', '-0.019239000000000006', '0.260624', '-0.293577', '0.19897499999999999', '0.277653', '0.164552', '0.40649', '-0.227907', '-0.400316'], ['0.151841', '0.07100100000000001', '-0.306629', '-0.24564200000000003', '0.163437', '0.343526', '-0.635576', '-0.08061000000000007', '-0.281144', '

In [77]:
# import pandas as pd

# df = pd.read_csv(
#     filepath,
#     sep = ' ',
#     index_col = False,
#     names = [f"d{i}" for i in range(dim_v_full)],
#     # nrows = 1000,
#     # quotechar='~~~~~~'
# )
# print(len(df))
# df

In [78]:
# Eventually, though, pandas overloads the memory. Let's instead read it line by line and load it to numpy

# Check a sample of 10 sentences
glove_tokens = []

with open(filepath, 'r') as f:
    for i in range(10):
        line = f.readline()
        full_line_all = line.split(' ')
        full_line = [float(i) for i in full_line_all[1:]]
        token = full_line_all[0]
        if i == 0:
            print(len(full_line))
            glove_vectors = np.array([full_line])
        else:
            glove_vectors = np.append(glove_vectors, [np.array(full_line)], axis = 0)
        glove_tokens.append(token)

print(len(glove_tokens), glove_tokens)
print(len(glove_vectors), glove_vectors)
print(glove_vectors.shape)
# print([len(i) for i in glove_vectors])
glove_vectors[:5]

50
10 ['the', ',', '.', 'of', 'to', 'and', 'in', 'a', '"', '-']
10 [[-3.833960e-01 -4.806740e-01 -2.741410e-01  1.326970e-01  6.488500e-02
  -9.248900e-02 -2.991000e-03  1.689180e-01  4.276900e-01  3.206200e-02
   6.006520e-01  4.555000e-03  2.215680e-01 -1.068170e-01  3.399550e-01
  -2.481650e-01 -6.553400e-02  1.650310e-01 -9.891500e-02 -2.670600e-01
  -3.516120e-01 -5.958360e-01 -7.483280e-01 -2.621910e-01 -4.001830e-01
   3.064770e-01  1.324700e-01  1.495320e-01 -4.629820e-01  4.364060e-01
  -3.284060e-01 -1.122970e-01 -3.181580e-01 -5.861770e-01 -2.417460e-01
  -3.828590e-01 -4.337860e-01  5.955205e+00  1.372390e-01 -6.451600e-02
   4.565220e-01 -1.923900e-02  2.606240e-01 -2.935770e-01  1.989750e-01
   2.776530e-01  1.645520e-01  4.064900e-01 -2.279070e-01 -4.003160e-01]
 [ 1.518410e-01  7.100100e-02 -3.066290e-01 -2.456420e-01  1.634370e-01
   3.435260e-01 -6.355760e-01 -8.061000e-02 -2.811440e-01  1.024800e-02
  -1.007730e-01  4.125000e-02 -2.845030e-01  4.505700e-01 -4.459460e

array([[-3.833960e-01, -4.806740e-01, -2.741410e-01,  1.326970e-01,
         6.488500e-02, -9.248900e-02, -2.991000e-03,  1.689180e-01,
         4.276900e-01,  3.206200e-02,  6.006520e-01,  4.555000e-03,
         2.215680e-01, -1.068170e-01,  3.399550e-01, -2.481650e-01,
        -6.553400e-02,  1.650310e-01, -9.891500e-02, -2.670600e-01,
        -3.516120e-01, -5.958360e-01, -7.483280e-01, -2.621910e-01,
        -4.001830e-01,  3.064770e-01,  1.324700e-01,  1.495320e-01,
        -4.629820e-01,  4.364060e-01, -3.284060e-01, -1.122970e-01,
        -3.181580e-01, -5.861770e-01, -2.417460e-01, -3.828590e-01,
        -4.337860e-01,  5.955205e+00,  1.372390e-01, -6.451600e-02,
         4.565220e-01, -1.923900e-02,  2.606240e-01, -2.935770e-01,
         1.989750e-01,  2.776530e-01,  1.645520e-01,  4.064900e-01,
        -2.279070e-01, -4.003160e-01],
       [ 1.518410e-01,  7.100100e-02, -3.066290e-01, -2.456420e-01,
         1.634370e-01,  3.435260e-01, -6.355760e-01, -8.061000e-02,
        -

In [79]:
# Now let's do it for more and more rows
glove_tokens = []

with open(filepath, 'r') as f:
    for i in range(10000):
        line = f.readline()
        full_line_all = line.strip().split(' ')
        full_line = [float(i) for i in full_line_all[1:]]
        token = full_line_all[0]
        if i == 0:
            print(len(full_line))
            glove_vectors = np.array([full_line])
        else:
            glove_vectors = np.append(glove_vectors, [np.array(full_line)], axis = 0)
        glove_tokens.append(token)


print(len(glove_tokens), glove_tokens[:5])
print(len(glove_vectors), glove_vectors[:5])
print(glove_vectors.shape)
# print([len(i) for i in glove_vectors])
glove_vectors

50
10000 ['the', ',', '.', 'of', 'to']
10000 [[-3.833960e-01 -4.806740e-01 -2.741410e-01  1.326970e-01  6.488500e-02
  -9.248900e-02 -2.991000e-03  1.689180e-01  4.276900e-01  3.206200e-02
   6.006520e-01  4.555000e-03  2.215680e-01 -1.068170e-01  3.399550e-01
  -2.481650e-01 -6.553400e-02  1.650310e-01 -9.891500e-02 -2.670600e-01
  -3.516120e-01 -5.958360e-01 -7.483280e-01 -2.621910e-01 -4.001830e-01
   3.064770e-01  1.324700e-01  1.495320e-01 -4.629820e-01  4.364060e-01
  -3.284060e-01 -1.122970e-01 -3.181580e-01 -5.861770e-01 -2.417460e-01
  -3.828590e-01 -4.337860e-01  5.955205e+00  1.372390e-01 -6.451600e-02
   4.565220e-01 -1.923900e-02  2.606240e-01 -2.935770e-01  1.989750e-01
   2.776530e-01  1.645520e-01  4.064900e-01 -2.279070e-01 -4.003160e-01]
 [ 1.518410e-01  7.100100e-02 -3.066290e-01 -2.456420e-01  1.634370e-01
   3.435260e-01 -6.355760e-01 -8.061000e-02 -2.811440e-01  1.024800e-02
  -1.007730e-01  4.125000e-02 -2.845030e-01  4.505700e-01 -4.459460e-01
  -5.871500e-02 -6

array([[-0.383396, -0.480674, -0.274141, ...,  0.40649 , -0.227907,
        -0.400316],
       [ 0.151841,  0.071001, -0.306629, ..., -0.251198, -0.092533,
         0.407064],
       [ 0.254684, -0.190782, -0.413487, ..., -0.033272,  0.018548,
        -0.015527],
       ...,
       [ 0.884323, -1.678937, -0.229539, ...,  0.153485,  0.014116,
        -0.590421],
       [-0.261274,  0.536087,  0.250557, ..., -0.504741,  0.18847 ,
         0.997212],
       [ 0.581784,  0.272695, -0.578221, ..., -0.154236, -0.24763 ,
        -0.418743]])

## Final read and proc

In [80]:
with open(filepath, 'r') as f:
    content = f.readlines()

content = [i.rstrip() for i in content]

print(len(content))
print(content[-2:])
print('--')
content[:5]

1291147
['zyuganovs 0.35429299999999997 -1.430418 0.021514000000000005 -0.20090800000000003 -0.9455359999999999 -0.161489 0.107792 -0.10722399999999999 0.261909 -0.47960200000000003 0.05404700000000001 -0.42511200000000005 0.8030900000000001 -0.20871699999999999 -0.32280200000000003 0.974184 0.645783 0.293382 -0.37169399999999997 0.255547 0.735328 0.07776200000000001 0.626626 0.10317399999999999 -0.439273 0.49276600000000004 -0.04701499999999999 0.40947100000000003 0.16487600000000002 0.08399999999999996 -0.04125599999999996 -0.160516 0.15443600000000002 -0.40825500000000003 -0.577035 -0.20102199999999998 -0.40166 -0.272561 -0.13721400000000006 -0.39441499999999996 0.211452 0.089304 0.09534499999999999 -0.069669 -0.10885599999999998 -0.9179039999999999 0.203786 0.433873 0.185676 0.24291100000000002', '<unk> 0.046781 0.136388 0.044058 0.009423 -0.0005650000000000099 0.0038280000000000002 0.12226100000000001 0.06209500000000001 0.047272999999999996 -0.021615999999999996 -0.22900600000000

['the -0.383396 -0.480674 -0.27414099999999997 0.132697 0.06488499999999997 -0.092489 -0.0029909999999999937 0.168918 0.42769 0.032062000000000035 0.600652 0.0045550000000000035 0.221568 -0.106817 0.339955 -0.24816500000000002 -0.06553399999999998 0.16503099999999998 -0.098915 -0.26706 -0.351612 -0.595836 -0.748328 -0.262191 -0.400183 0.306477 0.13247 0.149532 -0.462982 0.436406 -0.328406 -0.11229700000000001 -0.318158 -0.586177 -0.24174600000000002 -0.38285899999999995 -0.433786 5.955204999999999 0.137239 -0.064516 0.456522 -0.019239000000000006 0.260624 -0.293577 0.19897499999999999 0.277653 0.164552 0.40649 -0.227907 -0.400316',
 ', 0.151841 0.07100100000000001 -0.306629 -0.24564200000000003 0.163437 0.343526 -0.635576 -0.08061000000000007 -0.281144 0.01024799999999998 -0.100773 0.041249999999999995 -0.284503 0.45056999999999997 -0.44594599999999995 -0.058714999999999996 -0.06081500000000001 0.006295999999999996 -0.35699400000000003 -0.10028200000000001 0.270828 0.40327899999999994 

# Data proc

## glove_tokens

In [81]:
glove_tokens = [i.split(' ')[0] for i in content]
print(len(glove_tokens))
print(glove_tokens[:5])

1291147
['the', ',', '.', 'of', 'to']


In [82]:
print(glove_tokens[-5:])

['zyprex', 'zyro', 'zyrphelis', 'zyuganovs', '<unk>']


In [83]:
glove_tokens.index('king')

703

## glove_vectors

In [84]:
glove_vectors = [i.split(' ')[1:] for i in content]
glove_vectors = [[float(i) for i in j] for j in glove_vectors]
print(len(glove_vectors))
glove_vectors[0]

1291147


[-0.383396,
 -0.480674,
 -0.27414099999999997,
 0.132697,
 0.06488499999999997,
 -0.092489,
 -0.0029909999999999937,
 0.168918,
 0.42769,
 0.032062000000000035,
 0.600652,
 0.0045550000000000035,
 0.221568,
 -0.106817,
 0.339955,
 -0.24816500000000002,
 -0.06553399999999998,
 0.16503099999999998,
 -0.098915,
 -0.26706,
 -0.351612,
 -0.595836,
 -0.748328,
 -0.262191,
 -0.400183,
 0.306477,
 0.13247,
 0.149532,
 -0.462982,
 0.436406,
 -0.328406,
 -0.11229700000000001,
 -0.318158,
 -0.586177,
 -0.24174600000000002,
 -0.38285899999999995,
 -0.433786,
 5.955204999999999,
 0.137239,
 -0.064516,
 0.456522,
 -0.019239000000000006,
 0.260624,
 -0.293577,
 0.19897499999999999,
 0.277653,
 0.164552,
 0.40649,
 -0.227907,
 -0.400316]

# Get top 5 closest words

In [111]:
def get_top_5_closest_words(word):
    print('---------------------------------')
    # Get index
    what_index = glove_tokens.index(word)
    # Get glove embedding
    what_embedding = glove_vectors[what_index]
    # Get cosine similarity with other vectors
    cosine_similarity_list = []
    for index, i in enumerate(glove_vectors[:10000]): # we slice the first 10k words to reduce the runtime and just show how it works
        cosine_similarity_value = cosine_similarity([what_embedding], [i])
        cosine_similarity_list.append(float(cosine_similarity_value[0][0]))
    # Get top 5 matches
    ind = np.argpartition(cosine_similarity_list, -5)[-5:]
    top5 = np.array(cosine_similarity_list)[ind]
    top5 = sorted(top5)
    print(f'Top 5 words similar to "{word}"')
    for i in top5[::-1]:
        indx = cosine_similarity_list.index(i)
        print(f"- {glove_tokens[indx]} (cosine similarity: {i:.3f})")
    print('---------------------------------')
    return None

for i in ['queen', 'toy', 'giraffe']:
    get_top_5_closest_words(i)

---------------------------------
Top 5 words similar to "queen"
- queen (cosine similarity: 1.000)
- princess (cosine similarity: 0.855)
- lady (cosine similarity: 0.841)
- king (cosine similarity: 0.810)
- elizabeth (cosine similarity: 0.797)
---------------------------------
---------------------------------
Top 5 words similar to "toy"
- toy (cosine similarity: 1.000)
- toys (cosine similarity: 0.895)
- makers (cosine similarity: 0.748)
- shoe (cosine similarity: 0.744)
- shop (cosine similarity: 0.744)
---------------------------------
---------------------------------
Top 5 words similar to "giraffe"
- elephant (cosine similarity: 0.626)
- cat (cosine similarity: 0.550)
- fur (cosine similarity: 0.543)
- sheep (cosine similarity: 0.537)
- cats (cosine similarity: 0.533)
---------------------------------
